In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/medicine-descrptions/Task.csv
/kaggle/input/medicine-descrptions/TASK.xlsx
/kaggle/input/extractive-summarization/Extractivie_Summary.xlsx


In [2]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116


In [3]:
!pip install transformers

In [4]:
# Importing dependencies from transformers
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

In [6]:
# Load tokenizer 
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

In [7]:
# Load model 
model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

In [8]:
data=pd.read_excel('../input/medicine-descrptions/TASK.xlsx')
data

,TEST DATASET,Unnamed: 1
0,NaN,Introduction
1,NaN,Acnesol Gel is an antibiotic that fights bacte...
2,NaN,Ambrodil Syrup is used for treating various re...
3,NaN,Augmentin 625 Duo Tablet is a penicillin-type ...
4,NaN,Azithral 500 Tablet is an antibiotic used to t...
...,...,...
996,NaN,Azapure Tablet belongs to a group of medicines...
997,NaN,Arimidex 1mg Tablet is used alone or with oth...
998,NaN,Arpimune ME 100mg Capsule is used to prevent y...
999,NaN,Amlodac CH Tablet is a combination medicine us...


In [9]:
data.columns

Index(['TEST DATASET', 'Unnamed: 1'], dtype='object')

In [12]:
data.rename(columns = {'Unnamed: 1' : 'Introduction' }, inplace=True)
# Deleting the first row
data.drop(0)

,TEST DATASET,Introduction
1,NaN,Acnesol Gel is an antibiotic that fights bacte...
2,NaN,Ambrodil Syrup is used for treating various re...
3,NaN,Augmentin 625 Duo Tablet is a penicillin-type ...
4,NaN,Azithral 500 Tablet is an antibiotic used to t...
5,NaN,Alkasol Oral Solution is a medicine used in th...
...,...,...
996,NaN,Azapure Tablet belongs to a group of medicines...
997,NaN,Arimidex 1mg Tablet is used alone or with oth...
998,NaN,Arpimune ME 100mg Capsule is used to prevent y...
999,NaN,Amlodac CH Tablet is a combination medicine us...


In [13]:
# Converting the DataFrame into a dictionary
text= {}
for i in range(1,len(data['TEST DATASET'])):
    text[i] = data['Introduction'][i]
    
print(text[1])

Acnesol Gel is an antibiotic that fights bacteria. It is used to treat acne, which appears as spots or pimples on your face, chest or back. This medicine works by attacking the bacteria that cause these pimples.Acnesol Gel is only meant for external use and should be used as advised by your doctor. You should normally wash and dry the affected area before applying a thin layer of the medicine. It should not be applied to broken or damaged skin. Avoid any contact with your eyes, nose, or mouth. Rinse it off with water if you accidentally get it in these areas. It may take several weeks for your symptoms to improve, but you should keep using this medicine regularly. Do not stop using it as soon as your acne starts to get better. Ask your doctor when you should stop treatment.Common side effects like minor itching, burning, or redness of the skin and oily skin may be seen in some people. These are usually temporary and resolve on their own. Consult your doctor if they bother you or do not

In [19]:
def textSummarization(test_text):
    tokens = tokenizer(test_text, truncation=True, padding="longest", return_tensors="pt")
    summary = model.generate(**tokens)
    return summary[0]

In [17]:
summary_dict={}
for key in range(1,30):
    para=text[key]
    final_summary=textSummarization(para)
    decoded_summary=tokenizer.decode(final_summary)
    summary_dict[key]=decoded_summary
    if key==1 or key<6:
        print("Summary of the article - ",key)
        print(summary_dict[key])
print("Summarization is completed:-)")

Summary of the article -  1
Acnesol Gel is used to treat acne, which appears as spots or pimples on your face, chest or back.
Summary of the article -  2
Ambrodil Syrup is used for treating various respiratory tract disorders associated with excessive mucus.
Summary of the article -  3
Augmentin 625 Duo Tablet is a penicillin-type of antibiotic that helps your body fight infections caused by bacteria.
Summary of the article -  4
Azithral 500 Tablet is an antibiotic used to treat various types of bacterial infections of the respiratory tract, ear, nose, throat, lungs, skin, and eye in adults and children.
Summary of the article -  5
Alkasol Oral Solution is a medicine used in the treatment of gout and kidney stones.
Summarization is completed:-)


**Saving the summary in a table**

In [25]:
summary_table = pd.DataFrame(list(summary_dict.items()),columns = ['TEST DATASET','Summary'])

In [26]:
data_table = pd.DataFrame(list(text.items()),columns = ['TEST DATASET','Introduction'])

In [27]:
result  = pd.concat([data_table , summary_table['Summary']], axis = 1 , sort = False)

In [28]:
result.to_csv("Summary_File.csv")

# **ROUGE Metric**

In [29]:
!pip install rouge

In [30]:
from rouge import Rouge
ROUGE = Rouge()

In [32]:
from tabulate import tabulate

In [33]:
# Load summary results for a random data
candidate = [
    summary_dict[1],summary_dict[4],summary_dict[6],summary_dict[10],summary_dict[26]  # document 1: hypothesis
    ]
reference =[
    'Acnesol Gel is an antibiotic used to treat acne,which appears as spots or pimples on your face, chest or back, This medicine works by attacking the bacteria that cause these pimples.',
    'Azithral 500 antibiotic used to treat various types of bacterial infections,Azithral 500 Tablet is a broad-spectrum type of antibiotic effective in killing many types of gram-positive bacteria',
    'Allegra 120mg Tablet belongs to a group of medicines called antihistamines,Allegra 120mg Tablet should be takenÂ on an empty stomach.',
    'Aciloc RD 20 Tablet is a prescription medicine used to treat gastroesophageal reflux disease,Aciloc RD 20 Tablet is taken without food in a dose and duration as advised by the doctor. The dose you are given will depend on your condition and how you respond to the medicine.',
    'Atarax 10mg Tablet is an antihistamine used to treat symptoms of skin allergy,Atarax 10mg Tablet should be taken with or without food.'
   ]

In [34]:
#Calculating the average rouge score
Score={}
Score=ROUGE.get_scores(candidate, reference,avg=True)
rouge1=Score["rouge-1"]
rouge2=Score["rouge-2"]
rougel=Score["rouge-l"]

In [35]:
print(rouge1,rouge2,rougel,end='')

{'r': 0.43598444839824146, 'p': 0.546897546897547, 'f': 0.4717130531060797} {'r': 0.3103030303030303, 'p': 0.4379912424740011, 'f': 0.35353291783345886} {'r': 0.41780263021642333, 'p': 0.532082732082732, 'f': 0.4553865224938348}

In [36]:
board_dict = {
    'ROUGE-1': {
        'precision': rouge1['p'],
        'Recall':rouge1['r'] ,
        'f-score':rouge1['f']
    },
    'ROUGE-2': {
        'precision': rouge2['p'],
        'Recall':rouge2['r'] ,
        'f-score':rouge2['f']
    },
    'ROUGE-l': {
         'precision': rougel['p'],
        'Recall':rougel['r'] ,
        'f-score':rougel['f']
    }    
}


In [37]:
dframe = pd.DataFrame(board_dict)
print(tabulate(dframe.T, headers="keys"))

           precision    Recall    f-score
-------  -----------  --------  ---------
ROUGE-1     0.546898  0.435984   0.471713
ROUGE-2     0.437991  0.310303   0.353533
ROUGE-l     0.532083  0.417803   0.455387
